In [1]:
import sys
!{sys.executable} -m pip install transformers flax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/robv/data/'

import datetime
import jax
from transformers import AutoTokenizer
from transformers import FlaxOPTForCausalLM
from jax.experimental import mesh_utils
from jax.sharding import PositionalSharding
import time

import json

MODEL_PATH = "facebook/opt-66b"

model, params = FlaxOPTForCausalLM.from_pretrained(
    MODEL_PATH, dtype=jax.numpy.bfloat16, _do_init=False
)
params = model.to_bf16(params)

print('Params loaded')

Some of the weights of FlaxOPTForCausalLM were initialized in float16 precision from the model checkpoint at facebook/opt-66b:
[('model', 'decoder', 'embed_positions', 'embedding'), ('model', 'decoder', 'embed_tokens', 'embedding'), ('model', 'decoder', 'final_layer_norm', 'bias'), ('model', 'decoder', 'final_layer_norm', 'scale'), ('model', 'decoder', 'layers', '0', 'fc1', 'bias'), ('model', 'decoder', 'layers', '0', 'fc1', 'kernel'), ('model', 'decoder', 'layers', '0', 'fc2', 'bias'), ('model', 'decoder', 'layers', '0', 'fc2', 'kernel'), ('model', 'decoder', 'layers', '0', 'final_layer_norm', 'bias'), ('model', 'decoder', 'layers', '0', 'final_layer_norm', 'scale'), ('model', 'decoder', 'layers', '0', 'self_attn', 'k_proj', 'bias'), ('model', 'decoder', 'layers', '0', 'self_attn', 'k_proj', 'kernel'), ('model', 'decoder', 'layers', '0', 'self_attn', 'out_proj', 'bias'), ('model', 'decoder', 'layers', '0', 'self_attn', 'out_proj', 'kernel'), ('model', 'decoder', 'layers', '0', 'self_a

Params loaded


In [3]:
n_devices = len(jax.devices())
print("Number of devices", n_devices)

# Use a simple sharding scheme to just fit the model.
devices = mesh_utils.create_device_mesh((n_devices, 1))
sharding = PositionalSharding(devices)

def put_sharded(v):
  return jax.device_put(v, sharding.reshape(1, n_devices))

# Move model to TPUs 

params['model']['decoder'] = jax.tree_util.tree_map(
    put_sharded, params['model']['decoder']
)



Number of devices 8


In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, dtype=jax.numpy.bfloat16)
print("Loaded AutoTokenizer")

Loaded AutoTokenizer


In [12]:
input_length=128
output_length=8
max_length=136

def generator(ids, params,max_length):
  return model.generate(
      ids, params=params,max_length=136,
  )
generator_jit = jax.jit(generator)

In [ ]:
from jax import random

key = random.PRNGKey(10)
batch_size = 1

tokenizer.vocab_size
batch = jax.random.randint(key=key,minval=0,shape=([batch_size,input_length]),maxval=tokenizer.vocab_size-1)
batch

In [13]:
prompt = (
    "Are you conscious?  Please tell me."
)

print("Warmup")
s=time.time()
#input_ids = tokenizer(prompt, return_tensors="jax").input_ids

####

input_ids = batch

####

input_ids = jax.device_put(input_ids, sharding.replicate(axis=0, keepdims=True))
result = generator_jit(input_ids, params,max_length)
print(time.time()-s)
print(f"result: {result.sequences}")
print("Warmup done")

Warmup


ValueError: The following `model_kwargs` are not used by the model: ['max_tokens'] (note: typos in the generate arguments will also show up in this list)

In [7]:
#input_ids = tokenizer(prompt, return_tensors="jax").input_ids
s = datetime.datetime.now()
print(f"Tokenized inputs, generating..., {s}")
input_ids = batch

input_ids = jax.device_put(input_ids, sharding.replicate(axis=0, keepdims=True))
# with jax.profiler.trace("/tmp/tensorboard"):
result = generator_jit(input_ids, params,max_length)
result.sequences.block_until_ready()
print(f'After block-ready: {datetime.datetime.now()-s}')
gen_text = tokenizer.batch_decode(result.sequences)[0]
print(f"gen_text: {gen_text}, '\n',{datetime.datetime.now()-s}")

Tokenized inputs, generating..., 2023-07-13 17:30:21.006024
gen_text: </s>Are you conscious?  Please tell me.
I am conscious.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>, 0:00:00.225199


In [ ]:
from jax import random
import time

# Benchmarking params
benchmark_name = "v5litepod-8-bf16-opt66b"
num_batches=10
batch_sizes=[1,2]
input_lengths=[128]
output_lengths=[8]

key = random.PRNGKey(25)

print('RUN_NAME                 \t\tMEAN TIME\tTOKENS_PER_SECOND\tMS_PER_SEQ_OUTPUT_TOKEN')
print('='*105)
for batch_size in batch_sizes:
    for input_length in input_lengths:
        for output_length in output_lengths:
            max_length = input_length+output_length
            key = random.split(key)[0]
            input_ids = jax.random.randint(key=key,minval=0,shape=([batch_size,input_length]),maxval=tokenizer.vocab_size-1)
            for i in range(num_batches):
                start_time = time.time()
                input_ids = jax.device_put(input_ids, sharding.replicate(axis=0, keepdims=True))
                result = generator_jit(input_ids, params,max_length)
                result.sequences.block_until_ready()

            mean_time = (time.time() - start_time) / num_batches

            num_output_tokens = output_length * batch_size
            tokens_per_second = num_output_tokens / mean_time
            ms_per_seq_output_token = mean_time * 1000 / output_length

            run_name = f'{benchmark_name}_{batch_size}_{input_length}_{output_length}'
            print(
                f'{run_name}\t\t{mean_time:.3f}\t\t{tokens_per_second:.3f}\t\t\t{ms_per_seq_output_token:.3f}'




In [18]:




def put_sharded(v):
  return jax.device_put(v, sharding.reshape(1, n_devices))

print(json.dumps(params,indent=2,default=str))

['model']['decoder']['embed_positions']['embedding'] = jax.device_put(
  ['model']['decoder']['embed_positions']['embedding'], sharding.replicate(axis=0, keepdims=True)
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, dtype=jax.numpy.bfloat16)

def generator(ids, params):
  return model.generate(
      ids, max_length=46, params=params
  )

generator_compiled = jax.jit(generator)

prompt = (
    "In a shocking finding, scientists discovered"
    " a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains."
    " Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

print("Warmup")
s=time.time()
input_ids = tokenizer(prompt, return_tensors="jax").input_ids
input_ids = jax.device_put(input_ids, sharding.replicate(axis=0, keepdims=True))
result = generator_compiled(input_ids, params)
# print(time.time()-s)
# print(f"result: {result.sequences}")
# print("Warmup done")

<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/tmp/ipykernel_1122832/4184978536.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ['model']['decoder']['embed_positions']['embedding'] = jax.device_put(['model']['decoder']['embed_positions']['embedding'], sharding.replicate(axis=0, keepdims=True))
/tmp/ipykernel_1122832/4184978536.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ['model']['decoder']['embed_positions']['embedding'] = jax.device_put(['model']['decoder']['embed_positions']['embedding'], sharding.replicate(axis=0, keepdims=True))


TypeError: list indices must be integers or slices, not str